In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

from pypfopt import EfficientFrontier, risk_models, expected_returns
from pypfopt import plotting


In [6]:

# ==============================
# CONFIG
# ==============================

tickers = ["AAPL", "MSFT", "GOOGL"]
start_date = "2020-01-01"

# ==============================
# DATA
# ==============================

prices = yf.download(tickers, start=start_date)["Close"]
returns = prices.pct_change().dropna()


[*********************100%***********************]  3 of 3 completed

3 Failed downloads:
['MSFT', 'GOOGL', 'AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [ ]:

# ==============================
# BASIC INDIVIDUAL STATS
# ==============================

stats = pd.DataFrame(index=tickers)

stats["Mean Daily Return"] = returns.mean()
stats["Variance"] = returns.var()
stats["Std Dev"] = returns.std()
stats["Annual Volatility"] = returns.std() * np.sqrt(252)
stats["Cumulative Return"] = (1 + returns).prod() - 1
stats["Sharpe (rf=0)"] = stats["Mean Daily Return"] / stats["Std Dev"]

# Advanced distribution stats
stats["Skewness"] = returns.apply(skew)
stats["Kurtosis"] = returns.apply(kurtosis)

# Max Drawdown
cum_returns = (1 + returns).cumprod()
rolling_max = cum_returns.cummax()
drawdown = cum_returns / rolling_max - 1
stats["Max Drawdown"] = drawdown.min()

print("\n=== INDIVIDUAL ASSET STATS ===")
print(stats)


In [ ]:

# ==============================
# ADVANCED STATS
# ==============================

cov_matrix = returns.cov()
corr_matrix = returns.corr()

# Equal-weight portfolio as benchmark
equal_weights = np.ones(len(tickers)) / len(tickers)
portfolio_returns = returns @ equal_weights

# Betas vs portfolio
betas = returns.apply(lambda x: np.cov(x, portfolio_returns)[0, 1] /
                                  np.var(portfolio_returns))

# VaR and CVaR (95%)
var_95 = returns.quantile(0.05)
cvar_95 = returns[returns.le(var_95)].mean()

print("\n=== BETAS vs EQUAL-WEIGHT PORTFOLIO ===")
print(betas)

print("\n=== VaR 95% ===")
print(var_95)

print("\n=== CVaR 95% ===")
print(cvar_95)


In [ ]:

# ==============================
# PORTFOLIO OPTIMIZATION
# ==============================

mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# Max Sharpe
ef_sharpe = EfficientFrontier(mu, S)
weights_sharpe = ef_sharpe.max_sharpe()
perf_sharpe = ef_sharpe.portfolio_performance()

# Min Volatility
ef_minvol = EfficientFrontier(mu, S)
weights_minvol = ef_minvol.min_volatility()
perf_minvol = ef_minvol.portfolio_performance()

print("\n=== MAX SHARPE PORTFOLIO ===")
print(weights_sharpe)
print("Performance:", perf_sharpe)

print("\n=== MIN VOL PORTFOLIO ===")
print(weights_minvol)
print("Performance:", perf_minvol)


In [ ]:

# ==============================
# EFFICIENT FRONTIER PLOT
# ==============================

ef = EfficientFrontier(mu, S)

fig, ax = plt.subplots()

# Frontier
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True)

# Mark key portfolios
ax.scatter(perf_sharpe[1], perf_sharpe[0], marker="*", s=200, label="Max Sharpe")
ax.scatter(perf_minvol[1], perf_minvol[0], marker="X", s=200, label="Min Vol")

ax.set_title("Efficient Frontier")
ax.legend()
plt.show()